In [1]:
import os
import time

import json
import openai
import shortuuid
from dotenv import load_dotenv
import pandas as pd
import sys

parent_dir = os.path.abspath("..")
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from path import DATA_PROCESSED_COHORT_DIR, DATA_PROCESSED_DOCUMENTS_DIR

load_dotenv()
import random

from jinja2 import Template

openai.api_key = os.getenv("OPEN_API_KEY")

models = openai.Model.list()
list(filter(lambda x: x != None, map(lambda x: x.get("root") if "gpt" in x.get("root") else None, models["data"])))

['gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-4-0314',
 'gpt-3.5-turbo',
 'gpt-4-0613',
 'gpt-3.5-turbo-0301',
 'gpt-4',
 'gpt-3.5-turbo-0613']

In [2]:
openai.api_key = os.getenv("OPEN_API_KEY")

PROMPT_TEMPLATE = Template(
    """
Please generate a medical note using the following patient information as if you were a physician in the emergency department.

Patient Name: {{ patient_name }}
Age: {{ age }}
Gender: {{ gender }}
Chief Complaint: {{ chief_complaint }}

Return the response in JSON using the schema below. Do not return the schema itself or any other text.

```json
{
  "$schema": "http://json-schema.org/draft-04/schema#",
  "type": "object",
  "properties": {
    "history_of_present_illness": {
      "type": "string",
      "description": "The history of present illness should include a chronological description of the progression of the patient's present illness from the first sign and symptom to the present."
    },
    "review_of_symptoms": {
      "type": "object",
      "properties": {
        "constitutional": {
          "type": "string"
        },
        "heent": {
          "type": "string"
        },
        "cardiovascular": {
          "type": "string"
        },
        "respiratory": {
          "type": "string"
        },
        "gi": {
          "type": "string"
        },
        "gu": {
          "type": "string"
        },
        "musculoskeletal": {
          "type": "string"
        },
        "skin": {
          "type": "string"
        },
        "neurologic": {
          "type": "string"
        }
      },
      "required": [
        "constitutional",
        "heent",
        "cardiovascular",
        "respiratory",
        "gi",
        "gu",
        "musculoskeletal",
        "skin",
        "neurologic"
      ]
    },
    "past_medical_history": {
      "type": "string"
    },
    "medications": {
      "type": "string"
    },
    "past_surgical_history": {
      "type": "string"
    },
    "family_history": {
      "type": "string"
    },
    "social_history": {
      "type": "string"
    }
  },
  "required": [
    "history_of_present_illness",
    "review_of_symptoms",
    "past_medical_history",
    "medications",
    "past_surgical_history",
    "family_history",
    "social_history"
  ]
}
```
"""
)

MODEL_VERSION = "gpt-3.5-turbo-0613"

def call_openai_document_complete(
    fake_pt_name, fake_pt_age, fake_pt_gender, fake_chief_complaint
):
    """
    Call the OpenAI API to generate a document using the default template and a fake patient name
    """
    print(f"Using model {MODEL_VERSION}")
    return openai.ChatCompletion.create(
        model=MODEL_VERSION,
        messages=[
            {
                "role": "system",
                "content": PROMPT_TEMPLATE.render(
                    patient_name=fake_pt_name,
                    age=fake_pt_age,
                    gender=fake_pt_gender,
                    chief_complaint=fake_chief_complaint,
                ),
            }
        ],
    )

In [3]:
from collections import deque
from functools import lru_cache
from typing import Optional

class Chat:
    def __init__(self, patient_name: str, age: int, gender: str, chief_complaint: str, converstion_limit: int = 20,):
        
        # number of chats to remember
        self.messages_queue = deque(maxlen=converstion_limit)
        self.patient_name = patient_name
        self.age = age
        self.gender = gender
        self.chief_complaint = chief_complaint

    # @lru_cache(maxsize=512)
    def chat(self, message: Optional[str] = None) -> str:
        print("Using Model Version: ", MODEL_VERSION)
        initial_prompt = {
            "role": "system",
            "content": PROMPT_TEMPLATE.render(
                patient_name=self.patient_name,
                age=self.age,
                gender=self.gender,
                chief_complaint=self.chief_complaint,
            ),
        }
        if message:
            self.messages_queue.append({"role": "user", "content": message})

        response = openai.ChatCompletion.create(
            model=MODEL_VERSION, messages=[initial_prompt, *self.messages_queue]
        )

        reply = response["choices"][0]["message"].content

        self.messages_queue.append({"role": "assistant", "content": reply})

        return reply

In [4]:
class MissingFieldsError(Exception):
    def __init__(self, message, missing):            
        # Call the base class constructor with the parameters it needs
        super().__init__(message)
            
        # Now for your custom code...
        self.missing = missing
        
def parse_response(chat_completion: str) -> dict:
    content = None
    try:
        content = json.loads(chat_completion)
    except Exception as e:
        # pull of first and last line, gpt sometimes response with a leading ```json and ends with ```
        tmp = chat_completion.split("\n")
        while "{" not in tmp[0]:
            tmp = tmp[1:]
        while "}" not in tmp[-1]:
            tmp = tmp[:-1]
        tmp = "".join(tmp)
        content = json.loads(tmp)
    finally:
        empty_fields = list(map(lambda y: y[0], filter(lambda x: x[1] == "", content.items()))) + list(map(lambda y: "review_of_symptoms" + "." + y[0], filter(lambda x: x[1] == "", content.get("review_of_symptoms", {
            "constitutional": "",
            "cardiovascular": "",
            "respiratory": "",
            "gi": "",
            "gu": "",
            "musculoskeletal": "",
            "skin": "",
            "neurologic": ""
          }).items())))
        if len(empty_fields) > 0:
            raise MissingFieldsError(message="The response was invalid as following fields were left blank", missing=empty_fields)
    return content


In [5]:
MODEL_VERSION = "gpt-3.5-turbo-0613"

In [6]:
c = Chat("Aaron Gathers" , 38, "M", "Shortness of Breath")

In [17]:
def fetch_and_validate(chatInstance) -> str:
    res = chatInstance.chat()
    attempts = 0
    MAX_ATTEMPTS = 3
    while attempts < MAX_ATTEMPTS:
        try:
            valid_response = parse_response(res)
            return json.dumps(valid_response)
        except ValueError as e:
            print(f"Error: The response was invalid as the result did not contain valid JSON: {e}")
            res = chatInstance.chat(f"Error: The response was invalid as the result did not contain valid JSON: {e}")
        except MissingFieldsError as e:
            print(f"Error: The response was invalid as following fields were left blank: [{', '.join(e.missing)}]. Please fill in the empty fields.")
            res = chatInstance.chat(f"Error: The response was invalid as following fields were left blank: [{', '.join(e.missing)}]. Please fill in the empty fields.")
        finally:
            attempts = attempts + 1
    # return last attempt
    print(f"Unable to meet all requirements with {MAX_ATTEMPTS} attempts, result may have missing fields or be invalid")
    return res # string, may not be parsable dict

In [18]:
n = fetch_and_validate(c)

Using Model Version:  gpt-3.5-turbo-0613


In [20]:
parse_response(n)

{'history_of_present_illness': 'The patient, Aaron Gathers, is a 38-year-old male who presents to the emergency department with a chief complaint of shortness of breath. He reports that the shortness of breath started approximately 2 hours ago and has been progressively worsening. He denies any associated chest pain, cough, or fever. The shortness of breath is exacerbated by exertion and relieved by sitting up. He also notes that he feels slightly dizzy when standing up. He denies any recent travel, exposure to sick contacts, or known allergies. There is no history of similar episodes in the past.',
 'review_of_symptoms': {'constitutional': 'No fever, chills, or unintentional weight loss.',
  'heent': 'No headache, blurred vision, or changes in hearing or taste.',
  'cardiovascular': 'No chest pain, palpitations, or edema.',
  'respiratory': 'Shortness of breath is the primary symptom. No cough or wheezing.',
  'gi': 'No abdominal pain, nausea, vomiting, or changes in bowel movements.'